In [ ]:
import imaging_behavior_functions
import odor_behavioral_analysis
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import optimize
import scipy.io
import os
import mat73
from sklearn.preprocessing import StandardScaler
from scipy.interpolate import interp1d
from scipy.stats import iqr
from PIL import Image
import cv2
from IPython.display import display, clear_output
import time
import seaborn as sns
from scipy.stats import iqr
from scipy.ndimage import gaussian_filter1d
import glob
from scipy.spatial.distance import cdist
from sklearn.cluster import KMeans

In [ ]:
base_path = "C:/Users/wilson/OneDrive - Harvard University/Thesis - Wilson lab/2P imaging/preprocessed data/qualified_sessions/one_trial_sessions/"
example_path_data = base_path+"20230814-1_EPGhDeltaB_syntGCAMP7f_odor_apple_patchy/data/"
example_path_results = base_path+"20230814-1_EPGhDeltaB_syntGCAMP7f_odor_apple_patchy/results/"
trial_num = 1
odor_threshold = 5
time_interval_threshold = 16  # Assuming time is in seconds or an equivalent unit
k = 8
window_size = 30

is_mat73, roi_df, dff_raw, kinematics_raw, preprocessed_vars_ds, preprocessed_vars_odor = imaging_behavior_functions.load_intermediate_mat(example_path_data,trial_num)
behav_df = imaging_behavior_functions.make_df_behavior(dff_raw, preprocessed_vars_ds, preprocessed_vars_odor,trial_num,ball_d = 9)
xPos, yPos = imaging_behavior_functions.reconstruct_path(behav_df, ball_d = 9)
roi_names, hdeltab_index, epg_index, hdeltab_sequence, epg_sequence = imaging_behavior_functions.get_roi_seq(roi_df)
dff_all_rois, dff_time = imaging_behavior_functions.load_dff_raw(is_mat73, dff_raw)
neural_df = imaging_behavior_functions.make_df_neural(dff_all_rois, dff_time, roi_names, hdeltab_index, epg_index, hdeltab_sequence, epg_sequence)
behav_df, padded_result_df, smoothed_df = odor_behavioral_analysis.analysis_dfs(behav_df, time_interval_threshold, k, window_size)
combined_df = imaging_behavior_functions.combine_df(behav_df, neural_df)

In [13]:
smoothed_df

,time,fwV,sideV,yawV,heading,abssideV,absyawV,net_motion,net_motion_state,heading_adj,...,xPos,yPos,cluster_label_temporal,soft_cluster_label_temporal,cluster_label_spatial,fwV_smoothed,sideV_smoothed,yawV_smoothed,heading_smoothed,straightness_smoothed
0,0.000000,0.000000,0.000000,0.000000,0.385674,0.000000,0.000000,0.000000,0,0.385674,...,0.000000,0.000000,0,0,0,3.072275,-7.567584,1.376365,0.479128,0.761829
1,0.291806,6.425764,-16.468804,4.016872,0.509846,16.468804,4.016872,26.911439,0,0.509846,...,-0.065655,1.628857,0,0,0,3.208180,-7.953026,1.344198,0.500652,0.754700
2,0.583611,4.171223,-6.055598,-2.076583,0.502235,6.055598,2.076583,12.303404,0,0.502235,...,-0.032982,2.751727,0,0,0,3.353178,-8.263929,1.157035,0.537205,0.755254
3,0.875417,2.407029,-14.872266,8.301797,0.638375,14.872266,8.301797,25.581091,0,0.638375,...,-0.150991,3.258702,0,0,0,3.459225,-8.034249,0.717614,0.580443,0.732105
4,1.167222,4.021584,-1.169685,-4.926420,0.615807,1.169685,4.926420,10.117690,0,0.615807,...,0.093917,4.387152,0,0,0,3.666261,-7.146349,0.090427,0.623503,0.747510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6164,1798.689210,14.187531,-5.823839,-2.573166,5.321017,5.823839,2.573166,22.584536,0,-95.209948,...,-1187.363309,443.268740,0,0,6,14.515653,-7.755664,-1.762775,5.376011,NaN
6165,1798.981015,14.452897,-10.719389,-0.305870,5.203520,10.719389,0.305870,25.478156,0,-95.327445,...,-1191.402849,443.521638,0,0,6,13.310443,-9.264425,-1.193742,5.282641,NaN
6166,1799.272821,16.318690,-6.766734,-3.085316,5.133504,6.766734,3.085316,26.170740,0,-95.397461,...,-1196.056038,443.576894,0,0,6,11.714005,-10.401275,-0.466328,5.223468,NaN
6167,1799.564626,4.046410,-30.038337,1.817984,4.964697,30.038337,1.817984,35.902731,0,-95.566268,...,-1196.679118,443.002146,0,0,6,10.156611,-10.901462,0.257126,5.197676,NaN
